
<div dir=rtl >
<h1>قوانین انجمنی</h1>

قوانین انجمنی روابط و وابستگی‌های متقابل بین مجموعه بزرگی از اقلام داده‌ای را نشان می‌دهند. 

مثال متداول در رابطه با کشف قوانین انجمنی "تحلیل سبد خرید" است.  در این فرایند با توجه به اقلام مختلفی که مشتریان در سبد خریدشان قرار می‌دهند، عادات و رفتار خرید مشتریان مورد تحلیل قرار می‌گیرد و می‌توان با شناسایی ارتباط بین محصولات، الگوهای تکرار شونده در حین خرید را بدست آورد.

سه پارامتر مهم:
- Support  میزان محبوبیت یک مجموعه آیتم را با توجه به دفعات حضور در تراکنش‌ها نشان می‌دهد.
- Confidence  احتمال خرید آیتم y در صورت خرید آیتم x را نشان می‌دهد. x -> y
- Lift   ترکیب دو پارامتر بالا می‌باشد.

برای پیاده سازی قوانین انجمنی در این تمرین، از الگوریتم Apriori که یکی از محبوب‌ترین و کارآمدترین الگوریتم‌های این حوزه است استفاده می‌کنیم.


<font color='Green'> پرسش:  تاثیر مقادیر مختلف پارامتر Lift را در احتمال وقوع نتیجه بررسی کنید.  </font>


مقدار Lift معیاری برای اهمیت یک قانون است. با استفاده از فیلترهای قانون می توانید محدوده Lift مورد نظر را در تنظیمات تعریف کنیم.
Lift اساساً به ما می گوید که احتمال خرید دو محصول با هم 3.33 برابر بیشتر از احتمال خرید فقط یک محصول است. Lift=1 به این معنی است که هیچ ارتباطی بین محصولات A و B وجود ندارد. افزایش بیشتر از 1 به این معنی است که محصولات A و B با هم خریداری می شوند.

</div>





<div dir=rtl >
<h1>الگوریتم Apriori</h1>

روش کار الگوریتم به این صورت است که یک  حداقل مقدار پشتیبانی در نظر گرفته    می‌شود و تکرارها با مجموعه آیتم‌های مکرر اتفاق می‌افتند. در صورتیکه  مجموعه‌ها و زیرمجموعه‌ها مقدار پشتیبانی کمتر از حد آستانه داشته باشند حذف می‌شوند. این روند تا زمانیکه امکان حذف وجود نداشته باشد ادامه می‌یابد. 

در این بخش از تمرین می‌خواهیم الگوریتم apriori را بر روی مجموعه داده Hypermarket_dataset که شامل سفارشات خرید افراد از فروشگاه های مواد غذایی است اعمال کنیم.


</div>





<div dir=rtl >
<h1>آماده‌سازی داده</h1>

<font color='Green'>پرسش: برای شروع کار باید داده‌های دیتاست را در قالب ماتریس پراکنده که محصولات  خریداری شده در ستون‌ و شماره سفارشات خرید به عنوان شاخص (index) هستند آماده کنید. 


برای راحتی کار محصولات خریداری شده در هر سفارش را با اعداد 0,1 کدگذاری کنید.

 نمونه ماتریس خروجی شما:

<img src="https://drive.google.com/uc?id=1eD0jan1ZbeYqSklgK--ks7oeY-MyTA3p"></img>

</div>




In [8]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import association_rules

In [2]:
# your code here
# ...
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [42]:
Hypermarket_dataset = pd.read_csv("/content/gdrive/MyDrive/Hypermarket_dataset.csv")
Hypermarket_dataset = Hypermarket_dataset.dropna()
print(Hypermarket_dataset);

       Member_number        Date        itemDescription
0               1808  21-07-2015         tropical fruit
1               2552  05-01-2015             whole milk
2               2300  19-09-2015              pip fruit
3               1187  12-12-2015       other vegetables
4               3037  01-02-2015             whole milk
...              ...         ...                    ...
38760           4471  08-10-2014          sliced cheese
38761           2022  23-02-2014                  candy
38762           1097  16-04-2014               cake bar
38763           1510  03-12-2014  fruit/vegetable juice
38764           1521  26-12-2014               cat food

[38765 rows x 3 columns]


In [43]:
from mlxtend.preprocessing import TransactionEncoder

Hypermarket_dataset.itemDescription = Hypermarket_dataset.itemDescription.transform(lambda x: [x])
Hypermarket_dataset = Hypermarket_dataset.reset_index(drop=True)
Hypermarket_dataset = Hypermarket_dataset.groupby(['Member_number']).sum()['itemDescription']

encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(Hypermarket_dataset).transform(Hypermarket_dataset).astype("int"), columns=encoder.columns_)
transactions.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0



<div dir=rtl >
<h1>شناسایی الگوهای پر تکرار</h1>

<font color='Green'> پرسش:  با اعمال الگوریتم apriori و به ازای مقدار min_support = 0.07 تمامی الگوهای پر تکرار را تولید نمایید. 

</div>




In [45]:
# your code here
# ...

frq_items = apriori(transactions, min_support = 0.07, use_colnames = True)
print(frq_items)
  

     support                                    itemsets
0   0.078502                                  (UHT-milk)
1   0.119548                                      (beef)
2   0.079785                                   (berries)
3   0.158799                              (bottled beer)
4   0.213699                             (bottled water)
..       ...                                         ...
78  0.075680                    (yogurt, tropical fruit)
79  0.079785            (whole milk, whipped/sour cream)
80  0.150590                        (whole milk, yogurt)
81  0.082093  (whole milk, rolls/buns, other vegetables)
82  0.071832      (whole milk, yogurt, other vegetables)

[83 rows x 2 columns]



<div dir=rtl >
<h1>استخراج قوانین انجمنی</h1>


<font color='Green'> پرسش: تابعی بنویسید که با گرفتن دو ورودی confidence و lift قوانین انجمنی حاصل را در خروجی نشان دهد. </br>
برای دو حالت خروجی خود را در گزارش ثبت کنید.


</div>




In [65]:
# your code here
# ...
# Collecting the inferred rules in a dataframe

def show_rules(confidence , lift):
  rules = association_rules(frq_items, metric ="lift")
  rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
  filter_rules = rules[(rules['confidence'] >= confidence) & (rules['lift'] >= lift)]
  return filter_rules



rules1 = show_rules(0.37533,1.32643)
display(rules1)


print("\n")

rules2 = show_rules(0.55, 1)
display(rules2)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
93,"(whole milk, other vegetables)",(yogurt),0.19138,0.282966,0.071832,0.375335,1.326434,0.017678,1.14787


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
94,"(yogurt, other vegetables)",(whole milk),0.120318,0.458184,0.071832,0.597015,1.303003,0.016704,1.344507
88,"(rolls/buns, other vegetables)",(whole milk),0.146742,0.458184,0.082093,0.559441,1.220996,0.014859,1.229837
